In [0]:
import numpy as np
import os
import imutils
import time
import cv2
from google.colab.patches import cv2_imshow
import torch
from copy import deepcopy

In [0]:
def ReadCellImages(dir, imgSize):
  images = os.listdir(dir)
  images = [image for image in images if image.endswith('png')]

  X = np.empty([len(images),imgSize[0],imgSize[1],3], dtype=float)
  y = []

  count = 0
  for image in images:
    X[count] = cv2.imread(dir+image)#.reshape((1,-1))
    if '1' in image: y.append(0)
    elif '2' in image: y.append(1)
    elif '3' in image: y.append(2)
    count = count+1

  return (X,y)

In [0]:
train_data = ReadCellImages('cells/', (23,23))

In [0]:
def CreateBatch(data, index, batch_size):
  if index + batch_size > len(data[0]): batch_size = len(data[0]) - index - 1
  return (torch.cat(
      [torch.from_numpy(image).float().reshape(1, 3, 23, 23) for image in data[0][index:index + batch_size]], dim=0), 
      torch.tensor(data[1][index:index + batch_size], dtype=torch.long)
  )

In [0]:
class CNN_classifier(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv_model = torch.nn.Sequential(
        torch.nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.linear = torch.nn.Linear(in_features=32*10*10,out_features=3)

  def forward(self, batch):
    z = batch
    z = self.conv_model(z)
    z = z.view(-1, 32*10*10)
    return self.linear(z)

In [0]:
cnn = CNN_classifier()

In [0]:
optimizer = torch.optim.SGD(
    cnn.parameters(),
    lr=0.01
)

In [0]:
loss_builder = torch.nn.NLLLoss(reduction='mean')
m = torch.nn.LogSoftmax(dim=1)
s = torch.nn.Softmax(dim=1)

In [0]:
batch_size = 3
n_epochs = 10
dev_loss_list = []
dev_acc_list = []

dev_data = deepcopy(train_data)

for epoch in range(n_epochs):
  cnn.train()
  for i in range(0,len(train_data[0]),batch_size):
    batch = CreateBatch(train_data, i, batch_size)
    x = batch[0]
    gold = batch[1]
    y = cnn(x)
    loss = loss_builder(m(y),gold)

    cnn.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(cnn.parameters(), 5.)  # clip gradient if its norm exceed 5
    optimizer.step()

  cnn.eval()
  dev_acc = 0
  for i in range(0,len(dev_data[0]),1):
    batch = CreateBatch(dev_data, i, 1)
    x = batch[0]
    gold = batch[1]
    y = cnn(x)

    if np.max(s(y).detach().numpy()) < 0.9: gold_predicted = 3
    else: gold_predicted = np.argmax(s(y).detach().numpy())

    if gold_predicted == gold: dev_acc += 1

  dev_acc /= dev_data[0].shape[0]
  
  dev_loss_list.append(loss.item())
  dev_acc_list.append(dev_acc)

  print("Epoch: {:d}/{:d}".format(epoch+1,n_epochs))
  print ("Dev Avg Loss:", loss.item(), "\t\tDev Accurancy:", dev_acc)
  print()

Epoch: 1/10
Dev Avg Loss: 35.336421966552734 		Dev Accurancy: 0.6666666666666666

Epoch: 2/10
Dev Avg Loss: 8287.19140625 		Dev Accurancy: 0.3333333333333333

Epoch: 3/10
Dev Avg Loss: 1088.6676025390625 		Dev Accurancy: 0.3333333333333333

Epoch: 4/10
Dev Avg Loss: 267.56787109375 		Dev Accurancy: 0.3333333333333333

Epoch: 5/10
Dev Avg Loss: 16.49320411682129 		Dev Accurancy: 0.6666666666666666

Epoch: 6/10
Dev Avg Loss: 19.385438919067383 		Dev Accurancy: 0.6666666666666666

Epoch: 7/10
Dev Avg Loss: 0.36000004410743713 		Dev Accurancy: 0.6666666666666666

Epoch: 8/10
Dev Avg Loss: 0.3592735528945923 		Dev Accurancy: 0.6666666666666666

Epoch: 9/10
Dev Avg Loss: 0.35854873061180115 		Dev Accurancy: 0.6666666666666666

Epoch: 10/10
Dev Avg Loss: 0.35782548785209656 		Dev Accurancy: 0.6666666666666666



In [0]:
def pyramid(image, scale=1.5, minSize=(20, 20)):
	# yield the original image
	yield image
	# keep looping over the pyramid
	while True:
		# compute the new dimensions of the image and resize it
		w = int(image.shape[1] / scale)
		image = imutils.resize(image, width=w)
		# if the resized image does not meet the supplied minimum
		# size, then stop constructing the pyramid
		if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
			break
		# yield the next image in the pyramid
		yield image


def sliding_window(image, stepSize, windowSize):
	# slide a window across the image
	for y in range(0, image.shape[0], stepSize):
		for x in range(0, image.shape[1], stepSize):
			# yield the current window
			yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [0]:
# load the image and define the window width and height
image = cv2.imread('black_bubbles_1.png')
(winW, winH) = (23, 23)

In [0]:
cell_1_count = 0
cell_2_count = 0
cell_3_count = 0

# loop over the image pyramid
for resized in pyramid(image, scale=1.5):
	# loop over the sliding window for each layer of the pyramid
	for (x, y, window) in sliding_window(resized, stepSize=1, windowSize=(winW, winH)):
		# if the window does not meet our desired window size, ignore it
		if window.shape[0] != winH or window.shape[1] != winW:
			continue

		cnn.eval()
		window_reshaped = deepcopy(window)
		window_reshaped = window_reshaped.reshape((1,3,23,23))
		batch = CreateBatch((window_reshaped,[0,0,0]), 0, 1)
		patch = batch[0]
		gold = batch[1]
		pred = cnn(patch)
		if np.max(s(pred).detach().numpy()) >= 0.9:
			pred = np.argmax(s(pred).detach().numpy())
			if pred == 0: cell_1_count = cell_1_count + 1
			elif pred == 1: cell_2_count = cell_1_count + 1
			elif pred == 2: cell_3_count = cell_1_count + 1

		clone = resized.copy()
		cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
		# cv2_imshow(clone)
		cv2.waitKey(1)
		time.sleep(0.025)

In [0]:
print('Cell Type 1 Count', cell_1_count)
print('Cell Type 2 Count', cell_2_count)
print('Cell Type 3 Count', cell_3_count)